In [28]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore

cache_dir = LocalFileStore("./cache")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap= 50
)

loader = UnstructuredFileLoader("./files/something.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorestores = Chroma.from_documents(docs, cached_embeddings)

/Users/jeyeolbaek/Developer/study/fullstack_gpt/env/lib/python3.9/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/jeyeolbaek/Developer/study/fullstack_gpt/env/lib/python3.9/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


OpenAIChat(client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_kwargs={'frequency_penalty': 0, 'logit_bias': {}, 'max_tokens': 100, 'n': 1, 'presence_penalty': 0, 'request_timeout': None, 'temperature': 0.1, 'top_p': 1})

In [ ]:
vectorestores.similarity_search("something", 10)